In [3]:
import yfinance as yf
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv("/Users/francis/Desktop/Github/elon-musk-sentiment-stock/data/musk_quote_with_sentiment.csv")

In [ ]:
df['musk_quote_created_at'] = pd.to_datetime(df['musk_quote_created_at'])
print(df['musk_quote_created_at'])

0      2023-05-09 16:50:16+00:00
1      2023-05-09 16:12:06+00:00
2      2023-04-15 20:42:55+00:00
3      2023-04-12 19:07:08+00:00
4      2023-03-27 01:57:41+00:00
                  ...           
7268   2024-12-03 13:36:24+00:00
7269   2024-12-03 02:47:26+00:00
7270   2024-12-03 02:10:20+00:00
7271   2024-12-03 01:10:16+00:00
7272   2024-12-03 00:20:24+00:00
Name: musk_quote_created_at, Length: 7273, dtype: datetime64[ns, UTC]


In [83]:
ticker = "TSLA"
start_date = df["musk_quote_created_at"].min().date().isoformat()
end_date = df["musk_quote_created_at"].max().date().isoformat()
print(f"Start date: {start_date}")
print(f"End date: {end_date}")

Start date: 2015-06-27
End date: 2025-04-13


In [ ]:
tsla_df = yf.download(ticker, start=start_date, end=end_date)
tsla_df = tsla_df.reset_index()
tsla_df["Date"] = pd.to_datetime(tsla_df["Date"])
print(tsla_df["Date"])

[*********************100%***********************]  1 of 1 completed

0      2015-06-29
1      2015-06-30
2      2015-07-01
3      2015-07-02
4      2015-07-06
          ...    
2458   2025-04-07
2459   2025-04-08
2460   2025-04-09
2461   2025-04-10
2462   2025-04-11
Name: Date, Length: 2463, dtype: datetime64[ns]


In [101]:
df = df.reset_index(drop=True)
tsla_df = tsla_df.reset_index(drop=True)

# Convert multi-level to single-level columns
if tsla_df.columns.nlevels > 1:
    tsla_df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in tsla_df.columns]

In [132]:
print(tsla_df.columns)

Index(['Date_', 'Close_TSLA', 'High_TSLA', 'Low_TSLA', 'Open_TSLA',
       'Volume_TSLA'],
      dtype='object')


In [119]:
df['tweet_date'] = df['musk_quote_created_at'].dt.date
df['tweet_date'] = pd.to_datetime(df['tweet_date'])
merged_df = pd.merge(df, tsla_df[['Date_', 'Close_TSLA']], left_on='tweet_date', right_on='Date_', how='left')
print(merged_df.columns)

Index(['orig_tweet_id', 'orig_tweet_created_at', 'orig_tweet_text',
       'orig_tweet_url', 'orig_tweet_twitter_url', 'orig_tweet_username',
       'orig_tweet_retweet_count', 'orig_tweet_reply_count',
       'orig_tweet_like_count', 'orig_tweet_quote_count',
       'orig_tweet_view_count', 'orig_tweet_bookmark_count', 'musk_tweet_id',
       'musk_quote_tweet', 'musk_quote_retweet_count',
       'musk_quote_reply_count', 'musk_quote_like_count',
       'musk_quote_quote_count', 'musk_quote_view_count',
       'musk_quote_bookmark_count', 'musk_quote_created_at', 'vader_compund',
       'vader_label', 'vader_compound', 'tweet_date', 'musk_quote_date',
       'Date_', 'Close_TSLA'],
      dtype='object')


In [131]:
# First create a mapping of each trading day to its next trading day
next_trading_day = {}
trading_days = tsla_df['Date_'].sort_values().tolist()

for i in range(len(trading_days) - 1):
    next_trading_day[trading_days[i]] = trading_days[i+1]

# Apply this mapping to get the correct next day close
# First create a dict of dates to closing prices
date_to_close = dict(zip(tsla_df['Date_'], tsla_df['Close_TSLA']))

# Then apply the mapping correctly
df['tweet_date'] = pd.to_datetime(df['musk_quote_created_at']).dt.date
df['tweet_date'] = pd.to_datetime(df['tweet_date'])

# Create a proper merge
merged_df = pd.merge(df, tsla_df[['Date_', 'Close_TSLA']], 
                    left_on='tweet_date', 
                    right_on='Date_', 
                    how='left')

# Add next trading day
merged_df['next_trading_day'] = merged_df['Date_'].map(next_trading_day)
# Add next day's close using the mapping
merged_df['next_day_close'] = merged_df['next_trading_day'].map(date_to_close)

# Calculate percentage change
merged_df['pct_change_next_day'] = (merged_df['next_day_close'] - merged_df['Close_TSLA']) / merged_df['Close_TSLA']

# Check results
print(merged_df[['tweet_date', 'Date_', 'Close_TSLA', 'next_trading_day', 'next_day_close', 'pct_change_next_day']].head(10))

  tweet_date      Date_  Close_TSLA next_trading_day  next_day_close  \
0 2023-05-09 2023-05-09  169.149994       2023-05-10      168.539993   
1 2023-05-09 2023-05-09  169.149994       2023-05-10      168.539993   
2 2023-04-15        NaT         NaN              NaT             NaN   
3 2023-04-12 2023-04-12  180.539993       2023-04-13      185.899994   
4 2023-03-27 2023-03-27  191.809998       2023-03-28      189.190002   
5 2023-03-12        NaT         NaN              NaT             NaN   
6 2024-01-02 2024-01-02  248.419998       2024-01-03      238.449997   
7 2023-06-29 2023-06-29  257.500000       2023-06-30      261.769989   
8 2023-06-27 2023-06-27  250.210007       2023-06-28      256.239990   
9 2023-06-27 2023-06-27  250.210007       2023-06-28      256.239990   

   pct_change_next_day  
0            -0.003606  
1            -0.003606  
2                  NaN  
3             0.029689  
4            -0.013659  
5                  NaN  
6            -0.040134  
7      

In [128]:
merged_df.to_csv("/Users/francis/Desktop/Github/elon-musk-sentiment-stock/data/musk_tweets_with_tsla_return.csv", index=False)